## SRP : 단일 책임 원칙
## ocp : open close 

## 잔액 관리 인터페이스 작성

In [1]:
interface Balanceable {                    // 잔액을 처리하는 인터페이스 정의 
    fun credit(amount : Int)               // 입금 추상메소드 
    fun debit(amount : Int)                // 출금 추상메소드
    fun query() : String                   // 잔액 상태 조회 추상메소드 
    fun getBal() : Double                  // 잔액 조회 추상메소드 
}

## 클래스의 단일 책임으로 분리

In [2]:
class Balance(val accNo : String,                       // 잔액관리 클래스 
              var balance : Double) : Balanceable {     // 인터페이스 상속 및 구현 
    override fun credit(amount : Int) {                 // 입금 추성 메소드 구현 
        balance += amount.toDouble()
    }
    override fun debit(amount : Int) {                  // 출금 추상메소드 구현
        if (balance < amount) {                         // 잔액 체크 
            println("잔액이 부족합니다! $balance 이하로 출금해주세요.")
        } else {
            balance -= amount.toDouble() 
        }
    }
    override fun query() : String{                       // 잔액상태 추상메소드 구현 
         return "AccoNo= $accNo, Balance= $balance"
    }
    override fun getBal() = balance                      // 잔액조회 추상메소드 구현
}

class Agreement(val accNo :String,                       // 계약관리 클래스
                val productNo: String,
                var rate : Double)

class AgreementManager(val accNo :String,                 // 계약및 잔액관리 클래스 구현 
                       val balance : Balanceable) : 
                                  Balanceable by balance {// 잔액처리 클래스 위임처리 
                                                          // 클래스 위임처리로 내부 코드는 컴파일러가 생성
}
                                  
// 기능 추가 : Open Closed Principles                                
fun AgreementManager.calBenifit(rate : Double) {          // 이자 계산 후 입금처리는 확장함수로 구현 
    val benifit = balance.getBal() * rate /365            // 연이자로 계산 
    balance.credit(benifit.toInt())                       // 입금처리
}

## 계좌 및 잔액 생성 그 다음에 입출금 처리

In [3]:
val agree = Agreement("1", "1", 0.05)                      // 게약관리 객체 생성
                                                           
val agree_bal: Balanceable =                               // 상위 인터페이스 자료형에 할당 
                 Balance(agree.accNo, 0.0)                 // 잔액관리 객체 생성
val agreeMG  : AgreementManager =                          // 확장함수 사용을 위해 자기자료형에 할당
                 AgreementManager(agree.accNo,agree_bal)   // 계약잔액 매지저관리 객체 생성

agreeMG.credit(10000)                                      // 입금처리
println(agreeMG.query())                                   // 잔액조회

agreeMG.calBenifit(agree.rate)                             // 이자 계산 및 입금
println(agreeMG.query())                                   // 잔액조회 

agreeMG.debit(15000)                                       // 출금처리 : 잔액 초과 
agreeMG.debit(5000)                                        // 일부 출금
println(agreeMG.query())                                   // 잔액 조회 


AccoNo= 1, Balance= 10000.0
AccoNo= 1, Balance= 10001.0
잔액이 부족합니다! 10001.0 이하로 출금해주세요.
AccoNo= 1, Balance= 5001.0
